In [1]:
import numpy as np, pandas as pd
import statsmodels.api as sm
from scipy import stats
import random

# 모평균에 대한 검정

In [2]:
df_cats = pd.read_csv('forkedBigdataAnalystCert/part3/ch6/cats.csv').drop(columns='Unnamed: 0')
nar = np.array([random.gauss(mu = 10, sigma = 5) for i in range(0, 10)])

In [3]:
nar.mean(), nar.std()

(12.671773204838917, 5.099272377152198)

## normality check

In [4]:
stats.shapiro(nar)

ShapiroResult(statistic=0.9815852940671566, pvalue=0.9730836522814885)

In [5]:
stats.shapiro(df_cats['Bwt'])

ShapiroResult(statistic=0.9518791269479144, pvalue=6.730857622701027e-05)

p-value가 0.05 미만인 등 상황이면 데이터가 정규분포를 따르지 않는다고 봄.

## Z-tests

In [6]:
from scipy.stats import norm

In [7]:
stats.norm.cdf(1.96)

0.9750021048517795

In [8]:
H0mean = 10
KNOWNSIGMA = 5

In [9]:
ztest_statistic = (nar.mean() - H0mean)/(np.sqrt(len(nar)) * KNOWNSIGMA)
ztest_statistic, (1 - stats.norm.cdf(ztest_statistic))*2

(0.16897777437397166, 0.8658141234724337)

In [10]:
#문제는 음수인데! 그냥 norm.cdf가 양측검정.
stats.norm.cdf(ztest_statistic) * 2

1.1341858765275663

## One Sample T-Tests(정규성 만족 시)

In [11]:
from scipy.stats import ttest_1samp

In [12]:
stats.ttest_1samp(nar, popmean=9, alternative = 'greater') 
#popmean : H0가 되는 값. 기준값. ttest_1samp는 별도로 해당 값을 지정하는 args가 있음.

TtestResult(statistic=2.1601747856953075, pvalue=0.029525331590715784, df=9)

## Wilcoxon test(정규성 불만족시)

In [13]:
from scipy.stats import wilcoxon

In [14]:
H0mean = 2.1
stats.wilcoxon(df_cats['Bwt'] - H0mean, alternative = 'two-sided') #윌콕슨은 mu0를 데이터프레임에서 직접 빼 줘야 함!

WilcoxonResult(statistic=50.0, pvalue=2.765612175340855e-23)

In [15]:
stats.wilcoxon(df_cats['Bwt'] - H0mean, alternative = 'less')

WilcoxonResult(statistic=8995.0, pvalue=1.0)

# 한 모집단의 변화에 대한 검정

## Normality Check

In [16]:
df1 = pd.DataFrame({
    'before':[-2, -5, 13, -65, 36, 52, 16],
    'after':[8, 6, 6, 5, 8, 7, 3]
})
df2 = pd.DataFrame({
    'before':np.array([random.gauss(mu = 9, sigma = 5) for i in range(0, 10)]),
    'after':np.array([random.gauss(mu = 12, sigma = 5) for i in range(0, 10)])
})

In [17]:
df1['diff'] = df1['after'] - df1['before']
stats.shapiro(df1['diff'])

ShapiroResult(statistic=0.9230547083693623, pvalue=0.4934962434665802)

In [18]:
df2['diff'] = df2['after'] - df2['before']
stats.shapiro(df2['diff'])

ShapiroResult(statistic=0.9198434377525108, pvalue=0.35565095248467626)

## Paired sample t-tests(정규성 만족시)

사용 조건 : 한 집단에서 처치 전후 비교. 정규성 가정, 연속형 종속변수 한정 사용가능

In [19]:
from scipy.stats import ttest_rel

In [20]:
stats.ttest_rel(df2['before'], df2['after'], alternative='less')
#before과 after의 자리 바꿀 수 있음. 이 경우 less와 greater가 바뀔 수 있음. 
#앞자리가 뒷자리보다 less하다는 것이 alternative 'less'의 의미.

TtestResult(statistic=-0.9693310986042903, pvalue=0.17885334791139007, df=9)

## Wilcoxon Tests(정규성 불만족시)

In [21]:
from scipy.stats import wilcoxon

In [22]:
stats.wilcoxon(df1['after'], df1['before'], alternative='greater')

WilcoxonResult(statistic=12.0, pvalue=0.65625)

In [23]:
stats.wilcoxon(df1['diff'], alternative='greater')
#shapiro test의 'diff' column 활용해도 결과 동일.

WilcoxonResult(statistic=12.0, pvalue=0.65625)

# 두 모집단의 비교에 대한 검정

두 모집단 사이 '평균의 차이가 있는지' 검정. 

두 가지의 가정 존재. 각각 테스트하고, 만족 여부에 따라 총 3종류의 검정 사용
* 정규성을 만족하는가? -> 두 집단 중 하나라도 정규표본이 아니면 Mann-Whitney U tests 사용
* 등분산성을 만족하는가? -> equal_val 옵션이 달라짐.

In [24]:
a1 = df_cats[df_cats['Sex'] == 'M']['Bwt']
a2 = df_cats[df_cats['Sex'] == 'F']['Bwt']
b1 = [85, 90, 92, 88, 86, 89, 83, 87]
b2 = [80, 82, 88, 85, 84]
c1 = [85, 90, 92, 88, 86, 89, 83, 87]
c2 = [90, 72, 88, 85, 70]
#검정에 사용하는 자료는 두 그룹을 각각 벡터로 저장

## Normality Check

In [25]:
from scipy.stats import shapiro

In [26]:
stats.shapiro(a1), stats.shapiro(a2)

(ShapiroResult(statistic=0.9788323948987693, pvalue=0.11896203410780715),
 ShapiroResult(statistic=0.8909610476638475, pvalue=0.0003754219909760795))

In [27]:
stats.shapiro(b1), stats.shapiro(b2)

(ShapiroResult(statistic=0.9981893537736595, pvalue=0.999986994137081),
 ShapiroResult(statistic=0.9917398436295009, pvalue=0.9854182266624983))

In [28]:
stats.shapiro(c1), stats.shapiro(c2)

(ShapiroResult(statistic=0.9981893537736595, pvalue=0.999986994137081),
 ShapiroResult(statistic=0.8456276596537071, pvalue=0.1810888722063807))

## Homoscedasticity Check

In [29]:
from scipy.stats import levene

In [30]:
stats.levene(b1, b2, center = 'median')

LeveneResult(statistic=0.0027925869510027727, pvalue=0.958802951766629)

In [31]:
stats.levene(c1, c2, center = 'mean') #센터는 두 옵션 있음. 시험 기준에서는 중요하지 않다는 의미겠죠?

LeveneResult(statistic=24.594251901944208, pvalue=0.0004292905252180314)

## Independent Sample t-tests(정규성, 등분산성 만족시)

In [32]:
from scipy.stats import ttest_ind

In [33]:
stats.ttest_ind(b1, b2, alternative = 'less', equal_var = True)

TtestResult(statistic=2.2108140580092237, pvalue=0.9754257110537391, df=11.0)

## Welch Sample t-tests(정규성 만족시, 등분산성 불만족시)

In [34]:
from scipy.stats import ttest_ind

In [35]:
stats.ttest_ind(c1, c2, alternative = 'two-sided', equal_var = False)

TtestResult(statistic=1.5138518371143854, pvalue=0.19708101962250396, df=4.4812899054392625)

## Mann-Whitney U tests(정규성 불만족시)

In [36]:
from scipy.stats import mannwhitneyu

In [37]:
stats.mannwhitneyu(a1, a2, alternative = 'greater')

MannwhitneyuResult(statistic=3801.5, pvalue=4.100251117160876e-11)

# Testing for Distributions

* 샘플 10개 이하 : 비모수적 방법 사용
* 샘플 11~30개 : 샤피로-윌크, 콜모고로프-스미르노프 등 사용.
* 샘플 30개 이상 : CLT로 정규성 만족 가정 가능.

# ANOVA : 일원배치(one-way anova)

## stats 사용

In [48]:
from scipy.stats import f_oneway
df = pd.DataFrame({
    'A':[10.5, 11.3, 10.8, 9.6, 11.1, 10.2, 10.9, 11.4, 10.5, 10.3],
    'B':[11.9, 12.4, 12.1, 13.2, 12.5, 11.8, 12.2, 12.9, 12.4, 12.3],
    'C':[11.2, 11.7, 11.6, 10.9, 11.3, 11.1, 10.8, 11.5, 11.4, 11.0],
    'D':[9.8, 9.4, 9.1, 9.5, 9.6, 9.9, 9.2, 9.7, 9.3, 9.4]
})

In [50]:
stats.shapiro(df['A']), stats.shapiro(df['B']), stats.shapiro(df['C']), stats.shapiro(df['D'])

(ShapiroResult(statistic=0.9649054066073813, pvalue=0.8400161543468654),
 ShapiroResult(statistic=0.9468040874196029, pvalue=0.6308700692815115),
 ShapiroResult(statistic=0.9701646110856055, pvalue=0.892367306190296),
 ShapiroResult(statistic=0.9752339025839644, pvalue=0.9346854448707653))

In [51]:
stats.levene(df['A'], df['B'], df['C'], df['D'])

LeveneResult(statistic=1.9355354288758708, pvalue=0.14127835331346628)

In [ ]:
stats.f_oneway(df['A'], df['B'], df['C'], df['D'])

## statsmodels 활용

In [63]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
df = pd.read_csv('forkedBigdataAnalystCert/part3/ch2/fertilizer.csv')

In [64]:
model = ols('성장 ~ C(비료)', df).fit()
print(anova_lm(model))

            df    sum_sq    mean_sq          F        PR(>F)
C(비료)      3.0  43.21875  14.406250  89.126139  1.001838e-16
Residual  36.0   5.81900   0.161639        NaN           NaN


# ANOVA : 이원배치(two-way anova)

In [65]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
df = pd.read_csv('forkedBigdataAnalystCert/part3/ch2/tree.csv')

In [ ]:
model = ols('성장률 ~ C(비료) + C(나무) + C(비료):C(나무)', df).fit() 
#C() : labelencoding 범주형 자료를 나타내는 표시.
print(anova_lm(model))

                df       sum_sq      mean_sq          F        PR(>F)
C(비료)          2.0  1127.924259   563.962129   6.669256  1.857612e-03
C(나무)          3.0  4783.353938  1594.451313  18.855528  6.600012e-10
C(비료):C(나무)    6.0   717.520672   119.586779   1.414199  2.157357e-01
Residual     108.0  9132.639448    84.561476        NaN           NaN


typ = 1, 2, 3 존재. 기본값은 1.

# Chi-Squared tests

## 적합성 검정(Goodness-of-Fit tests)

1개의 범주형 자료가 특정 분포로부터 나온 것인지 확인하기 위해 사용

observed frequency(실제 데이터상 도수) vs. expected frequency(이론상 도수)

In [122]:
from scipy.stats import chisquare

In [124]:
observed_frequency = np.array([90, 160])
expected_frequency = np.array([0.45, 0.55]) * np.sum(num) 
#이론상 분포에 전체 표본의 수 곱해 이론상 각 범주의 기대도수 계산
stats.chisquare(f_obs = num, f_exp = expected)

Power_divergenceResult(statistic=8.181818181818182, pvalue=0.004231232899758152)

## 독립성 검정(Test of Independece)

In [39]:
from scipy.stats import chi2_contingency

In [40]:
df = pd.read_csv('forkedBigdataAnalystCert/part3/ch6/survey.csv"').drop(columns='Unnamed: 0')
df

,Sex,Wr.Hnd,NW.Hnd,W.Hnd,Fold,Pulse,Clap,Exer,Smoke,Height,M.I,Age
0,Female,18.5,18.0,Right,R on L,92.0,Left,Some,Never,173.0,Metric,18.250
1,Male,19.5,20.5,Left,R on L,104.0,Left,NaN,Regul,177.8,Imperial,17.583
2,Male,18.0,13.3,Right,L on R,87.0,Neither,NaN,Occas,NaN,NaN,16.917
3,Male,18.8,18.9,Right,R on L,NaN,Neither,NaN,Never,160.0,Metric,20.333
4,Male,20.0,20.0,Right,Neither,35.0,Right,Some,Never,165.0,Metric,23.667
...,...,...,...,...,...,...,...,...,...,...,...,...
232,Female,18.0,18.0,Right,L on R,85.0,Right,Some,Never,165.1,Imperial,17.667
233,Female,18.5,18.0,Right,L on R,88.0,Right,Some,Never,160.0,Metric,16.917
234,Female,17.5,16.5,Right,R on L,NaN,Right,Some,Never,170.0,Metric,18.583
235,Male,21.0,21.5,Right,R on L,90.0,Right,Some,Never,183.0,Metric,17.167


In [42]:
#crosstab으로 바꾸고 써야 함!
chi2_contingency(df['Sex'], df['Exer']) #error

TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
df_crosstab = pd.crosstab(df['Sex'], df['Exer'])
print(df_crosstab)
chi2_contingency(df_crosstab, correction = True) #연속성 수정을 하는 경우. 기본값은 True.

Exer    Freq  Some
Sex               
Female    49    58
Male      65    40


Chi2ContingencyResult(statistic=4.904232352768243, pvalue=0.026790957089770598, dof=1, expected_freq=array([[57.53773585, 49.46226415],
       [56.46226415, 48.53773585]]))

### 동일성 검정(Test of Homogeneity)

In [ ]:
df_crosstab = pd.crosstab(df['Sex'], df['Exer'])
print(df_crosstab)
stats.chi2_contingency(df_crosstab, correction = False) #연속성 수정을 하지 않는 경우

Exer    Freq  Some
Sex               
Female    49    58
Male      65    40


Chi2ContingencyResult(statistic=4.904232352768243, pvalue=0.026790957089770598, dof=1, expected_freq=array([[57.53773585, 49.46226415],
       [56.46226415, 48.53773585]]))

# StatsModels 사용법

In [47]:
from statsmodels.formula.api import ols, logit

In [61]:
lm = ols(formula='Height ~ Age + Pulse', data = df).fit()
lm.summary(), lm.rsquared, lm.rsquared_adj

(<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                 Height   R-squared:                       0.008
 Model:                            OLS   Adj. R-squared:                 -0.003
 Method:                 Least Squares   F-statistic:                    0.7161
 Date:                Thu, 20 Jun 2024   Prob (F-statistic):              0.490
 Time:                        03:55:54   Log-Likelihood:                -633.26
 No. Observations:                 171   AIC:                             1273.
 Df Residuals:                     168   BIC:                             1282.
 Df Model:                           2                                         
 Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
 ---------------------------------------------------------------------

In [74]:
newdata = pd.DataFrame({
    'Age' : [10],
    'Pulse' : [100]
})

In [77]:
lm.get_prediction(newdata).summary_frame(alpha = 0.05)

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,171.165659,2.178469,166.864957,175.466361,151.14151,191.189808


`statsmodels.formula.api`와 `statsmodels.api`의 차이 :

* `statsmodels.formula.api` : ols function. R 스타일 ols('y~x', data = df) 사용.
* `statsmodels.api` : OLS class. Python 스타일 OLS() 초기화 -> OLS(df_y, df_X).fit()으로 사용.

# Regression Analysis

## 상관계수에 대한 검정

## 단순선형회귀 검정

In [ ]:
from statsmodels.formula.api import ols

# GLM

## Logistic Regression

In [67]:
from statsmodels.formula.api import logit
df_logistic = pd.read_csv('forkedBigdataAnalystCert/part3/ch5/health_survey.csv')
df_logistic.head()

,age,bmi,smoker,activity_level,disease
0,62,35.179089,0,0,1
1,65,18.576042,0,2,1
2,71,33.178426,0,1,1
3,18,37.063007,1,2,0
4,21,17.613266,0,0,0


In [68]:
model_logistic = logit('disease ~ age + bmi', data = df_logistic).fit()
model_logistic.summary()

Optimization terminated successfully.
         Current function value: 0.643725
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                disease   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      997
Method:                           MLE   Df Model:                            2
Date:                Tue, 19 Nov 2024   Pseudo R-squ.:                 0.04996
Time:                        05:38:10   Log-Likelihood:                -643.72
converged:                       True   LL-Null:                       -677.58
Covariance Type:            nonrobust   LLR p-value:                 1.984e-15
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.8700      0.289     -6.482      0.000      -2.435      -1.305
age            0.0177      0.004      4.747      0.000       0.010       0.025
bmi            0.0563      0.009      6.418      0.000       0.039       0.074
==============================================================================
"""

In [76]:
pred = model_logistic.get_prediction({'age':30, 'bmi':10})
pred.summary_frame()

,predicted,se,ci_lower,ci_upper
0,0.315286,0.033978,0.252749,0.385317


In [135]:
np.exp(model_logistic.params['bmi'])

1.057950853075076

In [138]:
#잔차이탈도 : -2*로그가능도
-2*model_logistic.llf

1287.4492329364175

## Poisson Regression

In [ ]:
poi_model = sm.GLM(df_logistic['age'], df_logistic[['disease', 'bmi']], 
                   family = sm.families.Poisson()).fit()
#기본 설정 link function : log 

In [149]:
poi_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                    age   No. Observations:                 1000
Model:                            GLM   Df Residuals:                      998
Model Family:                 Poisson   Df Model:                            1
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -29254.
Date:                Sun, 10 Nov 2024   Deviance:                       52873.
Time:                        18:50:47   Pearson chi2:                 1.20e+05
No. Iterations:                     6   Pseudo R-squ. (CS):         -7.907e+19
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
disease        0.5584      0.013     44.085      0.000       0.534       0.583
bmi            0.1138      0.000    331.647      0.000       0.113       0.115
==============================================================================
"""